# Advanced Linear Regression with Sklearn

## 📚 Learning Objectives
- Implement Linear Regression using proper pipelines
- Perform comprehensive model evaluation
- Analyze residuals and assumptions
- Compare multiple regression techniques
- Implement feature selection and engineering

## 🎯 What You'll Learn
1. Proper train-test splitting
2. Feature preprocessing with pipelines
3. Model training and evaluation
4. Residual analysis
5. Feature importance
6. Model comparison

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1️⃣ Load and Explore Data

In [ ]:
# Load dataset
df = pd.read_csv('data/dataset.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nColumn Names and Types:")
print(df.dtypes)
print(f"\nMissing Values:")
print(df.isnull().sum())
print(f"\nBasic Statistics:")
df.describe()

In [ ]:
# Separate features and target
target_col = 'median_house_value'
X = df.drop(columns=[target_col])
y = df[target_col]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nTarget statistics:")
print(y.describe())

## 2️⃣ Train-Test Split
**CRITICAL**: Always split BEFORE any preprocessing to prevent data leakage!

In [ ]:
# Split data (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"\nTraining set percentage: {X_train.shape[0]/df.shape[0]*100:.1f}%")
print(f"Test set percentage: {X_test.shape[0]/df.shape[0]*100:.1f}%")

## 3️⃣ Build Preprocessing Pipeline
Using pipelines ensures:
- No data leakage
- Reproducible preprocessing
- Easy deployment

In [ ]:
# Identify column types
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Numerical columns ({len(num_cols)}): {num_cols}")
print(f"Categorical columns ({len(cat_cols)}): {cat_cols}")

# Numerical pipeline: Impute median -> Scale
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical pipeline: Impute 'missing' -> OneHotEncode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])

print("\n✅ Preprocessing pipeline created successfully!")

## 4️⃣ Train Multiple Models
Let's compare different regression techniques

In [ ]:
# Define models to compare
models = {
    'Linear Regression': LinearRegression(),
    'Ridge (L2)': Ridge(alpha=1.0),
    'Lasso (L1)': Lasso(alpha=1.0),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5)
}

# Store results
results = {}
trained_pipelines = {}

# Train and evaluate each model
for name, model in models.items():
    print(f"\n{'='*60}")
    print(f"Training: {name}")
    print(f"{'='*60}")
    
    # Create pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # Train model
    pipeline.fit(X_train, y_train)
    
    # Make predictions
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Store results
    results[name] = {
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'predictions': y_test_pred
    }
    trained_pipelines[name] = pipeline
    
    # Print results
    print(f"\nTraining Metrics:")
    print(f"  RMSE: ${train_rmse:,.2f}")
    print(f"  MAE:  ${train_mae:,.2f}")
    print(f"  R²:   {train_r2:.4f}")
    
    print(f"\nTest Metrics:")
    print(f"  RMSE: ${test_rmse:,.2f}")
    print(f"  MAE:  ${test_mae:,.2f}")
    print(f"  R²:   {test_r2:.4f}")
    
    # Check for overfitting
    overfit_score = train_r2 - test_r2
    if overfit_score > 0.1:
        print(f"\n⚠️  Warning: Possible overfitting detected (gap: {overfit_score:.4f})")
    else:
        print(f"\n✅ Good generalization (gap: {overfit_score:.4f})")

## 5️⃣ Model Comparison Visualization

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame(results).T
comparison_df = comparison_df.drop('predictions', axis=1)

print("\n📊 Model Comparison Summary:")
print(comparison_df.round(2))

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# RMSE comparison
comparison_df[['train_rmse', 'test_rmse']].plot(kind='bar', ax=axes[0], color=['skyblue', 'coral'])
axes[0].set_title('RMSE Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylabel('RMSE ($)')
axes[0].set_xlabel('Model')
axes[0].legend(['Training', 'Test'])
axes[0].tick_params(axis='x', rotation=45)

# MAE comparison
comparison_df[['train_mae', 'test_mae']].plot(kind='bar', ax=axes[1], color=['lightgreen', 'salmon'])
axes[1].set_title('MAE Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylabel('MAE ($)')
axes[1].set_xlabel('Model')
axes[1].legend(['Training', 'Test'])
axes[1].tick_params(axis='x', rotation=45)

# R² comparison
comparison_df[['train_r2', 'test_r2']].plot(kind='bar', ax=axes[2], color=['gold', 'purple'])
axes[2].set_title('R² Score Comparison', fontsize=14, fontweight='bold')
axes[2].set_ylabel('R² Score')
axes[2].set_xlabel('Model')
axes[2].legend(['Training', 'Test'])
axes[2].tick_params(axis='x', rotation=45)
axes[2].axhline(y=0, color='r', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

## 6️⃣ Residual Analysis
Analyzing residuals helps us validate regression assumptions

In [ ]:
# Select best model for detailed analysis
best_model_name = max(results, key=lambda x: results[x]['test_r2'])
best_pipeline = trained_pipelines[best_model_name]
y_pred = results[best_model_name]['predictions']

print(f"\n🏆 Best Model: {best_model_name}")
print(f"Test R² Score: {results[best_model_name]['test_r2']:.4f}")

# Calculate residuals
residuals = y_test - y_pred

# Create residual plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Residuals vs Predicted Values
axes[0, 0].scatter(y_pred, residuals, alpha=0.5, edgecolors='k')
axes[0, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Predicted Values', fontsize=12)
axes[0, 0].set_ylabel('Residuals', fontsize=12)
axes[0, 0].set_title('Residual Plot', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# 2. Histogram of Residuals
axes[0, 1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Residuals', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title('Distribution of Residuals', fontsize=14, fontweight='bold')
axes[0, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[0, 1].grid(True, alpha=0.3)

# 3. Q-Q Plot
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Actual vs Predicted
axes[1, 1].scatter(y_test, y_pred, alpha=0.5, edgecolors='k')
axes[1, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                'r--', lw=2, label='Perfect Prediction')
axes[1, 1].set_xlabel('Actual Values', fontsize=12)
axes[1, 1].set_ylabel('Predicted Values', fontsize=12)
axes[1, 1].set_title('Actual vs Predicted', fontsize=14, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle(f'Residual Analysis - {best_model_name}', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Statistical tests
print("\n📈 Residual Statistics:")
print(f"Mean of residuals: {residuals.mean():.2f} (should be close to 0)")
print(f"Std of residuals: {residuals.std():.2f}")
print(f"Min residual: ${residuals.min():,.2f}")
print(f"Max residual: ${residuals.max():,.2f}")

## 7️⃣ Cross-Validation
More robust evaluation using k-fold cross-validation

In [ ]:
# Perform cross-validation for best model
cv_results = cross_validate(
    best_pipeline,
    X_train, y_train,
    cv=5,
    scoring=['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'],
    return_train_score=True
)

# Convert to RMSE
cv_rmse = np.sqrt(-cv_results['test_neg_mean_squared_error'])
cv_mae = -cv_results['test_neg_mean_absolute_error']
cv_r2 = cv_results['test_r2']

print(f"\n🔄 5-Fold Cross-Validation Results for {best_model_name}:")
print(f"\nRMSE per fold: {cv_rmse}")
print(f"Average RMSE: ${cv_rmse.mean():,.2f} (+/- ${cv_rmse.std() * 2:,.2f})")
print(f"\nMAE per fold: {cv_mae}")
print(f"Average MAE: ${cv_mae.mean():,.2f} (+/- ${cv_mae.std() * 2:,.2f})")
print(f"\nR² per fold: {cv_r2}")
print(f"Average R²: {cv_r2.mean():.4f} (+/- {cv_r2.std() * 2:.4f})")

# Visualize CV results
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].boxplot(cv_rmse)
axes[0].set_title('Cross-Validation RMSE', fontsize=14, fontweight='bold')
axes[0].set_ylabel('RMSE ($)')
axes[0].grid(True, alpha=0.3)

axes[1].boxplot(cv_mae)
axes[1].set_title('Cross-Validation MAE', fontsize=14, fontweight='bold')
axes[1].set_ylabel('MAE ($)')
axes[1].grid(True, alpha=0.3)

axes[2].boxplot(cv_r2)
axes[2].set_title('Cross-Validation R²', fontsize=14, fontweight='bold')
axes[2].set_ylabel('R² Score')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8️⃣ Feature Importance Analysis

In [ ]:
# Extract feature names after preprocessing
feature_names = num_cols.copy()

# Add one-hot encoded feature names
if len(cat_cols) > 0:
    ohe = best_pipeline.named_steps['preprocessor'].named_transformers_['cat']['onehot']
    cat_feature_names = ohe.get_feature_names_out(cat_cols)
    feature_names.extend(cat_feature_names)

# Get coefficients
coefficients = best_pipeline.named_steps['regressor'].coef_

# Create feature importance dataframe
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
}).sort_values('abs_coefficient', ascending=False)

print("\n🔍 Top 15 Most Important Features:")
print(feature_importance.head(15))

# Visualize top features
top_n = 15
top_features = feature_importance.head(top_n)

plt.figure(figsize=(12, 8))
colors = ['green' if x > 0 else 'red' for x in top_features['coefficient']]
plt.barh(range(top_n), top_features['coefficient'], color=colors, edgecolor='black')
plt.yticks(range(top_n), top_features['feature'])
plt.xlabel('Coefficient Value', fontsize=12)
plt.title(f'Top {top_n} Feature Importances - {best_model_name}', fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 📊 Key Takeaways

### Model Performance:
1. **Best Model**: Check the comparison above
2. **Generalization**: Look at train vs test scores
3. **Stability**: Cross-validation shows consistency

### Assumptions Check:
1. **Linearity**: Residual plot should show random scatter
2. **Normality**: Q-Q plot should be roughly linear
3. **Homoscedasticity**: Residuals should have constant variance
4. **Independence**: No patterns in residual plot

### Best Practices Applied:
✅ Proper train-test split BEFORE preprocessing
✅ Used pipelines to prevent data leakage
✅ Multiple model comparison
✅ Cross-validation for robust evaluation
✅ Residual analysis for assumption validation
✅ Feature importance analysis

### Next Steps:
1. Try polynomial features for non-linear relationships
2. Implement feature selection techniques
3. Tune hyperparameters (alpha for Ridge/Lasso)
4. Handle outliers if detected
5. Try ensemble methods for better performance